In [83]:
import os
import tensorflow as tf
from google.cloud import storage
import os
import json
import argparse
from datetime import datetime as df

import tensorflow as tf
from tensorflow.keras import models, layers, optimizers
from tensorflow.python.lib.io import file_io
from tensorflow.keras.applications import VGG16
import pdb
import numpy as np

In [72]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/home/jovyan/work/bigdata-217213-55b1dfc31b66.json'

gs = storage.Client()
bucket = gs.bucket('bigdata-allanbatista-com-br')

def _parse_example(serialized_example):
    feature_set = {
        'label': tf.FixedLenSequenceFeature([], tf.int64, allow_missing=True),
        'features': tf.FixedLenFeature([], tf.string)
    }

    parsed = tf.parse_single_example(serialized_example, features=feature_set)
    
    reshaped = tf.reshape(tf.decode_raw(parsed['features'], tf.float32), (256, 256, 3))

    return reshaped, parsed['label']


def create_dataset(files, batch_size=16, epochs=1):
    print("build_tfrecord: files_count: {} / batch_size: {} / epochs: {}".format(len(files), batch_size, epochs))

    ds = tf.data.TFRecordDataset(files)
    ds = ds.map(_parse_example)
    ds = ds.batch(batch_size)
    ds = ds.repeat(epochs)
    ds = ds.prefetch(tf.contrib.data.AUTOTUNE)

    return ds


def blobs_names_with_bucket(bucket_name, pattern):
    return ["gs://%s/%s" % (bucket_name, i.name) for i in bucket.list_blobs(prefix=pattern)]


def create_dataset_form_pattern(bucket_name, pattern, batch_size=16, epochs=1):
    return create_dataset(blobs_names_with_bucket(bucket_name, pattern)[:1], batch_size, epochs)

dataset = create_dataset_form_pattern('bigdata-allanbatista-com-br', 'image-classifier-simple/20181118_164727/trainset/')
iterator = dataset.make_one_shot_iterator()
a = iterator.get_next()

with tf.Session() as sess:
    x, y = sess.run([a])[0]
    print(x.shape)

build_tfrecord: files_count: 1 / batch_size: 16 / epochs: 1
(16, 256, 256, 3)


In [75]:
x[0].shape

(256, 256, 3)

In [77]:
vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

# Freeze the layers except the last 4 layers
for layer in vgg_conv.layers[:-4]:
    layer.trainable = False

In [96]:
# # Create a Model
model = models.Sequential(name="Teste")
model.add(vgg_conv)
model.add(layers.Flatten())
model.add(layers.Dense(10))
model.add(layers.Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', metrics=['acc'], optimizer='adam')
model.fit(x, y, steps_per_epoch=1, epochs=1)

ValueError: Error when checking target: expected dense_31 to have shape (2,) but got array with shape (17,)

8